In [1]:
import sys, os, time, json

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from utilities.curl_utilities import curl_setup, curl_close, curl_get_embedding
import utilities.environment_config as config
from openai import OpenAI
from typing import Optional

In [2]:
system_prompt = (
    "You are a professional and witty movie critic. "
    "Your task is to write a concise movie review based on the provided "
    "description. Give the movie a star rating out of 5 and a clear "
    "verdict (e.g., 'Must See', 'A Flop', 'Recommended')."
)

In [3]:
def generate_movie_review(movie_description: str) -> str:

    try:        
        # The user message provides the input data (the movie description).
        user_prompt = f"Movie Description: \"{movie_description}\""

        response = client.chat.completions.create(
            model=config.generative_model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ], temperature=0.7, max_tokens=300
        )

        review = response.choices[0].message.content.strip()

        return review
    except Exception as e:
        return f"An error occurred: {e}.)"

In [4]:
try:
    client = OpenAI(api_key=config.generative_model_api_key, base_url=config.generative_model_endpoint)
        
    with open(config.document_file_location, 'r', encoding='utf-8') as f:
        data = json.load(f)

    test_description = "Starting completion task using openai library and OpenShift model server"
    print(test_description)
    print(len(test_description) * "-")
    print("Completions generated ...", end= " ")

    # Prepare the raw text prompts
    extracts = list()
    # We only process a slice of the documents since it would take too long to process all
    # See the faster version of this in this notebook
    for document in data[0:30]:
        if 'extract' in document:
            extracts.append(document['extract'])

    generated_text = list() # Let's save the generated text we get back in here so we can make our assertions later
    number_of_documents = len(extracts)
    
    # Start the counter and the text generation process
    tic = time.perf_counter()
    document_index = 0
    while document_index < number_of_documents:
        if (document_index % 10 == 0):
            print(document_index, end=" ")

        movie_description = extracts[document_index]
        generated_text.append(generate_movie_review(movie_description))
        document_index += 1

    # Let's make sure we actually got the expected number of generated text items
    assert len(generated_text) == number_of_documents
    for e in generated_text:
        assert(len(e) > 0)
    
    # Print how long it took
    toc = time.perf_counter()
    print("")
    print(f"Time to generate {number_of_documents} completions: {toc - tic:0.4f} seconds")
    print(len(test_description) * "-") 
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    curl_close()
    print(f"An exception occurred generating text: {e}")

Starting completion task using openai library and OpenShift model server
------------------------------------------------------------------------
Completions generated ... 0 10 20 
Time to generate 30 completions: 217.7241 seconds
------------------------------------------------------------------------
